In [ ]:
from __future__ import print_function
import cv2
import argparse
import time
import numpy as np
from CentroidTracker import CentroidTracker, progressBar, showWindows, georeference
import params
from collections import defaultdict
from math import sqrt
import os
from vidstab import VidStab
import matplotlib.pyplot as plt

In [ ]:
# ---- input video file path
video = './input/DJI_0008_short.mp4'
video_stabilized = './repo/output_short_stab.avi' # Path to stabilized Video


path = defaultdict(list)

## Stabilize Video

In [ ]:
# Stabilize Video and save to File "video_stabilized"
stabilizer = VidStab(kp_method='FAST', threshold=42, nonmaxSuppression=False)
stabilizer.stabilize(input_path = video, output_path = video_stabilized, border_type = 'black', border_size=100)

stabilizer.plot_trajectory()
plt.show()

stabilizer.plot_transforms()
plt.show()


In [ ]:
# Set basic variables
ct = CentroidTracker()
current_frame = 0
id = 1
path = defaultdict(list)

In [ ]:
backSub = cv2.createBackgroundSubtractorMOG2()

In [ ]:
# Start time
start = time.time()

In [ ]:
capture = cv2.VideoCapture(video_stabilized)

# running the loop
# extracting the frames
ret, first_frame = capture.read()

# converting to gray-scale
prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)

In [ ]:
prev = cv2.goodFeaturesToTrack(prev_gray, mask=None, **params.feature_params)
mask = np.zeros_like(first_frame)
mask2 = np.zeros_like(first_frame)

In [ ]:
#Georeferencing
coord_pixel0_x = 0
coord_pixel0_y = 0
scale = 1

In [ ]:
while True:
    current_frame += 1
    ret, frame = capture.read()

#     if frame is None:
#         print('finished!')
#         break
    
    
    fgMask = backSub.apply(frame)
    # optimization of elements
    thresh = cv2.threshold(fgMask, params.threshold, 255, cv2.THRESH_BINARY)[1]
    se0 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    cleaned = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, se0)
    kernel_dil = np.ones(params.kernel_dilatation, np.uint8)
    thresh = cv2.dilate(thresh, kernel_dil, iterations=1)

    se1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (11, 11))
    se2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (41, 41))
    opened = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, se1)
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, se2)
    
    # detect the contours on the binary image using cv2.CHAIN_APPROX_NONE
    contours, hierarchy = cv2.findContours(image=closed, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    mask = np.zeros_like(first_frame)
    # draw contours on the original image
    contours = [x for x in contours if len(x) >= 30]
    cv2.drawContours(image=mask, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2,
                     lineType=cv2.LINE_AA)
    
    rects = []
    ROI_number = 0
    for cntr in contours:
        x, y, w, h = cv2.boundingRect(cntr)
        if params.min_width < w < params.max_width and params.min_height < h < params.max_height:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            ROI = mask[y:y + h, x:x + w]
            cv2.putText(mask, str(ROI_number), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))
            rects.append([x, y, x + w, y + h])
            ROI_number += 1
    
    
    # update centroid tracker using the computed set of bounding box rectangles
    objects, life = ct.update(rects)
    
    #build path in swiss coordinate system
    if objects is not None:
        for (objectID, centroid) in objects.items():
            if life[objectID] > params.min_life:
                # draw both the ID of the object and the centroid of the object on the output frame
                text = "ID {}".format(objectID)
                x = centroid[0]
                y = centroid[1]
                cv2.putText(frame, text, (x - 20, y - 20), cv2.FONT_HERSHEY_SIMPLEX, 100, (255, 255, 255), 50)
                cv2.circle(frame, (x, y), 4, (255, 255, 255), -1)
                if objectID in path.keys():
                    path[objectID].append((coord_pixel0_x+(x*scale), coord_pixel0_y-(y*scale)))
                else:
                    path[objectID] = [(coord_pixel0_x+(x*scale), coord_pixel0_y-(y*scale))]
                    
    # End time
    end = time.time()
    # Calculate frames per second
    fps = round(current_frame / (end-start), 2)
    
    
    cv2.imshow("output", mask)
#     imS = cv2.resize(mask, (1100, 800))                    # Resize image
#     cv2.imshow("output", imS)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

In [ ]:
#############----------- RUN ALL ABOVE ---------------